In [15]:
import numpy as np
from tqdm.notebook import tqdm
import trimesh
from queue import PriorityQueue
import matplotlib.pyplot as plt
from make_vtk import write_lines_to_vtk,write_points_to_vtk
from vtk_revise import read_vtk,write_vtk
from faster_generate_cone import generate_vectors_in_cone,plot_vectors_in_3d
from generate_interpolate_vector import generate_multiple_vectors

In [16]:
def normalize_points(points):
    """Normalize a set of 3D points."""
    norms = np.linalg.norm(points, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Avoid division by zero
    return points / norms

In [17]:
white = read_vtk('./AllCortexData/GS_CortexODE/SUBJ_001_MR_BL/surf/lh.CortexODE.white.vtk')
pial = read_vtk('./AllCortexData/GS_CortexODE/SUBJ_001_MR_BL/surf/lh.CortexODE.pial.vtk')

white_vertices = white['vertices']
white_faces = white['faces'][:,1:]

pial_vertices = pial['vertices']
pial_faces = pial['faces'][:,1:]

In [18]:
# Load your target mesh
origin_mesh = trimesh.Trimesh(vertices=white_vertices, faces=white_faces)
target_mesh = trimesh.Trimesh(vertices=pial_vertices, faces=pial_faces)

# origin vertex normals
origin_directions = origin_mesh.vertex_normals.copy()
white2pial_dir = normalize_points(pial_vertices - white_vertices)
dir_sign = (np.sum(white2pial_dir*origin_directions,axis=1) < 0)
origin_directions[dir_sign] *= -1

# print(white_vertices.shape)
# print(type(white2pial_dir))
# print(type(origin_directions))


In [4]:
# Pre-compute all the cone directions for all origins
all_origins = []

all_directions = generate_multiple_vectors(origin_directions,white2pial_dir , 50)
for idx in tqdm(range(len(white_vertices))):
    origin_pos = white_vertices[idx]
    all_origins.extend([origin_pos] * 50)
    
print(len(all_origins))
print(len(all_directions)) 
    

  0%|          | 0/142422 [00:00<?, ?it/s]

  0%|          | 0/142422 [00:00<?, ?it/s]

7121100
7121100


In [24]:

# print(origin_directions[0])
print(all_directions[99])
print(white2pial_dir[1])


# print(white2pial_dir[1])
# print(all_directions[99])

[ 0.06252081 -0.1446068  -0.98751205]
[ 0.06252081 -0.1446068  -0.98751205]


In [20]:
# Find ray-mesh intersections for all origins and directions at once
locations, index_ray, index_tri = target_mesh.ray.intersects_location(ray_origins=all_origins, ray_directions=all_directions)

In [21]:
# Now, process the results

threshold = 7.2

pairs = []
thicknesses = []
not_intersect = []
not_intersect_idx = []
too_long_pair = []

index_groups = {}
for i, ray_idx in tqdm(enumerate(index_ray), total=len(index_ray)):
    if ray_idx not in index_groups:
        index_groups[ray_idx] = []
    index_groups[ray_idx].append(i)

  0%|          | 0/24155061 [00:00<?, ?it/s]

In [22]:

# 'idx' is a factor to know which original vertex we are looking at. 
# This will change every 50 iterations (since there are 50 samples for each vertex).
for idx in tqdm(range(0, len(all_origins), 50)):

    # real_idx_1 = np.where((index_ray >= idx) & (index_ray < idx+50))
    real_idx_2 = []
    for i in range(idx, idx+50):
        real_idx_2.extend(index_groups.get(i, []))
        
    subset_locations = locations[real_idx_2]

    subset_origin = all_origins[idx]
    
    distances = [np.linalg.norm(loc - subset_origin) for loc in subset_locations]

    if distances:
        min_distance_index = np.argmin(distances)
        closest_location = subset_locations[min_distance_index]
        closest_distance = distances[min_distance_index]
        
        # pairs.append((subset_origin, closest_location))
        # thicknesses.append(closest_distance)
        
        if closest_distance > threshold:
            # thicknesses.append(float('nan'))
            too_long_pair.append((subset_origin, closest_location))
            not_intersect.append(subset_origin)
            real_idx = int(idx/50)
            not_intersect_idx.append(real_idx)
            white2pial_dist = np.linalg.norm(white_vertices[real_idx]-pial_vertices[real_idx])
            pairs.append((white_vertices[real_idx], pial_vertices[real_idx]))
            thicknesses.append(white2pial_dist)

            
        else : 
            pairs.append((subset_origin, closest_location))
            thicknesses.append(closest_distance)
    else:
        # thicknesses.append(float('nan'))
        # thicknesses.append()
        not_intersect.append(subset_origin)
        not_intersect_idx.append(int(idx/50))
        
        real_idx = int(idx/50)
        not_intersect_idx.append(real_idx)
        white2pial_dist = np.linalg.norm(white_vertices[real_idx]-pial_vertices[real_idx])
        pairs.append((white_vertices[real_idx], pial_vertices[real_idx]))
        thicknesses.append(white2pial_dist)

  0%|          | 0/142422 [00:00<?, ?it/s]

In [23]:
nan_count = np.isnan(thicknesses).sum()
thicknesses = np.array(thicknesses)
white['new_thickness'] = thicknesses
not_inter_ori_ray = []
for i in not_intersect_idx : 
    not_inter_ori_ray.append((white_vertices[i],pial_vertices[i]))


print(nan_count)
print(len(not_intersect))


print(max(thicknesses))
print(min(thicknesses))


write_vtk(white,f'./new_data/subject1_lh_white_inter.vtk')
write_lines_to_vtk(pairs, f"./new_data/pair_line_inter.vtk")
write_points_to_vtk(not_intersect,f"./new_data/not_intersect_point_inter.vtk",color=(255,0,0))
write_lines_to_vtk(too_long_pair, f"./new_data/long_line_inter.vtk")
write_lines_to_vtk(not_inter_ori_ray, f"./new_data/not_inter_ori_ray.vtk")


# write_vtk(white,f'./cone_data/subject1_lh_white_inter_thres{threshold}.vtk')
# write_lines_to_vtk(pairs, f"./cone_data/pair_line_inter_thres{threshold}.vtk")
# write_points_to_vtk(not_intersect,f"./cone_data/not_intersect_point_inter_thres{threshold}.vtk",color=(255,0,0))
# write_lines_to_vtk(too_long_pair, f"./cone_data/long_line_inter_{threshold}.vtk")
# write_lines_to_vtk(not_inter_ori_ray, f"./cone_data/not_inter_ori_ray.vtk")

19
38.36070613725069
0.0002492948700882892
